In [ ]:
#| default_exp handlers.ospar

# OSPAR 

> This data pipeline, known as a "handler" in Marisco terminology, is designed to clean, standardize, and encode [OSPAR data](https://odims.ospar.org/en/) into `NetCDF` format. The handler processes raw OSPAR data, applying various transformations and lookups to align it with `MARIS` data standards.

Key functions of this handler:

- **Cleans** and **normalizes** raw OSPAR data
- **Applies standardized nomenclature** and units
- **Encodes the processed data** into `NetCDF` format compatible with MARIS requirements

This handler is a crucial component in the Marisco data processing workflow, ensuring OSPAR data is properly integrated into the MARIS database.



Note: *Additionally, an optional encoder (pipeline) is provided below to process data into a `.csv` format compatible with the MARIS master database. This feature is maintained for legacy purposes, as data ingestion was previously performed using OpenRefine.*

:::{.callout-tip}

For new MARIS users, please refer to [Understanding MARIS Data Formats (NetCDF and Open Refine)](https://github.com/franckalbinet/marisco/tree/main/install_configure_guide) for detailed information.

:::

The present notebook pretends to be an instance of [Literate Programming](https://www.wikiwand.com/en/articles/Literate_programming) in the sense that it is a narrative that includes code snippets that are interspersed with explanations. When a function or a class needs to be exported in a dedicated python module (in our case `marisco/handlers/ospar.py`) the code snippet is added to the module using `#| exports` as provided by the wonderful [nbdev](https://nbdev.readthedocs.io/en/latest/) library.

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import pandas as pd 
import numpy as np
#from functools import partial 
import fastcore.all as fc 
from pathlib import Path 
#from dataclasses import asdict
from typing import List, Dict, Callable, Tuple, Any 
#from collections import OrderedDict, defaultdict
import re
#from functools import partial

from marisco.utils import (
    Remapper, 
    ddmm_to_dd,
    Match, 
    get_unique_across_dfs,
    NA
)

from marisco.callbacks import (
    Callback, 
    Transformer, 
    EncodeTimeCB, 
    AddSampleTypeIdColumnCB,
    AddNuclideIdColumnCB, 
    LowerStripNameCB, 
    SanitizeLonLatCB, 
    CompareDfsAndTfmCB, 
    RemapCB
)

from marisco.metadata import (
    GlobAttrsFeeder, 
    BboxCB, 
    DepthRangeCB, 
    TimeRangeCB, 
    ZoteroCB, 
    KeyValuePairCB
)

from marisco.configs import (
    nuc_lut_path, 
    nc_tpl_path, 
    cfg, 
    species_lut_path, 
    sediments_lut_path, 
    bodyparts_lut_path, 
    detection_limit_lut_path, 
    filtered_lut_path, 
    get_lut, 
    unit_lut_path,
    prepmet_lut_path,
    sampmet_lut_path,
    counmet_lut_path, 
    lab_lut_path,
    NC_VARS
)

from marisco.encoders import (
    NetCDFEncoder, 
)

from marisco.decoders import (
    nc_to_dfs,
    get_netcdf_properties, 
    get_netcdf_group_properties,
    get_netcdf_variable_properties
)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#| hide
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)  # Show full column width

## Configuration and file paths

1. **fname_in** - is the path to the folder containing the OSPAR data in CSV format. The path can be defined as a relative path. 

2. **fname_out_nc** - is the path and filename for the NetCDF output.The path can be defined as a relative path. 

3. **Zotero key** - is used to retrieve attributes related to the dataset from [Zotero](https://www.zotero.org/). The MARIS datasets include a [library](https://maris.iaea.org/datasets) available on [Zotero](https://www.zotero.org/groups/2432820/maris/library). 

4. **ref_id** - refers to the location in archive of the Zotero library.


In [ ]:
# | exports
fname_in = '../../_data/accdb/ospar/20241021/csv'
fname_out_nc = '../../_data/output/191-OSPAR-2024.nc'
zotero_key ='LQRA4MMK' # OSPAR MORS zotero key
ref_id = 191 # OSPAR reference id as defined by MARIS

## Load data

[OSPAR Environmental Monitoring Data](https://odims.ospar.org/en/) is provided as a Microsoft Access database. [`Mdbtools`](https://github.com/mdbtools/mdbtools) can be used to convert the tables of the Microsoft Access database to `.csv` files on Unix-like OS.

**Example steps**:

1. [Download data](https://odims.ospar.org/en/)
2. Install `mdbtools` via `VScode` Terminal (for instance):

    ```
    sudo apt-get -y install mdbtools
    ````

3. Install unzip via VScode Terminal 

    ```
    sudo apt-get -y install unzip
    ````

4. In `VS code` terminal (for instance), navigate to the marisco data folder

    ```
    cd /home/marisco/downloads/marisco/_data/accdb/ospar
    ```

5. Unzip `OSPAR_Env_Concentrations_20241021.zip`

    ```
    unzip OSPAR_Env_Concentrations_20241021.zip
    ```

6. Run `preprocess.sh` to generate the required data files

    ```
    ./preprocess.sh OSPAR_Env_Concentrations_20241021.zip
    ````

7. Content of `preprocess.sh` script:
    ```
    #!/bin/bash

    # Example of use: ./preprocess.sh OSPAR_Env_Concentrations_20241021.zip
    unzip $1
    dbname=$(ls *.accdb *.mdb)
    mkdir csv
    for table in $(mdb-tables -1 "$dbname"); do
        echo "Export table $table"
        mdb-export "$dbname" "$table" > "csv/$table.csv"
    done
    ```

Once converted to `.csv` files, the data is ready to be loaded into a dictionary of dataframes.
    

Load OSPAR data and return the data in a Python dictionary of dataframes with the dictionary key as the sample type.

In [ ]:
#| exports
default_smp_types = {'Seawater data': 'SEAWATER', 'Biota data': 'BIOTA'}

In [ ]:
#| exports
def load_data(src_dir:str, # Directory where the source CSV files are located
              lut:dict=default_smp_types # A dictionary with the file name as key and the sample type as value
              ) -> dict: # A dictionary with sample types as keys and their corresponding dataframes as values
    "Load `OSPAR` data and return the data in a dictionary of dataframes with the dictionary key as the sample type."
    return {
        sample_type: pd.read_csv(Path(src_dir) / f'{file_name}.csv', encoding='unicode_escape')
        for file_name, sample_type in lut.items()
    }

`dfs` includes a dictionary of dataframes that is created from the OSPAR dataset defined by `fname_in`. The data to be included in each dataframe is sorted by sample type. Each dictionary is defined with a key equal to the sample type. 

In [ ]:
#|eval: false
dfs = load_data(fname_in)
print('keys/sample types: ', dfs.keys())

for key in dfs.keys():
    print(f'{key} columns: ', dfs[key].columns)

keys/sample types:  dict_keys(['SEAWATER', 'BIOTA'])
SEAWATER columns:  Index(['ID', 'Contracting Party', 'RSC Sub-division', 'Station ID',
       'Sample ID', 'LatD', 'LatM', 'LatS', 'LatDir', 'LongD', 'LongM',
       'LongS', 'LongDir', 'Sample type', 'Sampling depth', 'Sampling date',
       'Nuclide', 'Value type', 'Activity or MDA', 'Uncertainty', 'Unit',
       'Data provider', 'Measurement Comment', 'Sample Comment',
       'Reference Comment'],
      dtype='object')
BIOTA columns:  Index(['ID', 'Contracting Party', 'RSC Sub-division', 'Station ID',
       'Sample ID', 'LatD', 'LatM', 'LatS', 'LatDir', 'LongD', 'LongM',
       'LongS', 'LongDir', 'Sample type', 'Biological group', 'Species',
       'Body Part', 'Sampling date', 'Nuclide', 'Value type',
       'Activity or MDA', 'Uncertainty', 'Unit', 'Data provider',
       'Measurement Comment', 'Sample Comment', 'Reference Comment'],
      dtype='object')


## Normalize nuclide names

### Lower & strip nuclide names

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Some nuclide names contain one or multiple trailing spaces.

:::

In [ ]:
#| eval: false
df = get_unique_across_dfs(load_data(fname_in), 'Nuclide', as_df=True, include_nchars=True)
df['stripped_chars'] = df['value'].str.strip().str.replace(' ', '').str.len()
print(df[df['n_chars'] != df['stripped_chars']])

    index        value  n_chars  stripped_chars
0       0  239, 240 Pu     11.0             9.0
2       2       99Tc        6.0             4.0
5       5          NaN      NaN             NaN
7       7      137Cs        7.0             5.0
11     11      210Po        7.0             5.0
12     12      99Tc         7.0             4.0


To fix this issue, we use the `LowerStripNameCB` callback. For each dataframe in the dictionary of dataframes, it corrects the nuclide name by converting it lowercase, striping any leading or trailing whitespace(s).

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripNameCB(col_src='Nuclide', col_dst='Nuclide')])
dfs_output=tfm()
for key, df in dfs_output.items():
    print(f'{key} Nuclides: ')
    print(df['Nuclide'].unique())
    
print(tfm.logs)

SEAWATER Nuclides: 
['137cs' '239,240pu' '226ra' '228ra' '99tc' '3h' '210po' '210pb' nan]
BIOTA Nuclides: 
['137cs' '226ra' '228ra' '239,240pu' '99tc' '210po' '210pb' '3h' 'cs-137'
 '238pu' '239, 240 pu' '241am']
["Convert 'Nuclide' column values to lowercase, strip spaces, and store in 'Nuclide' column."]


### Remap nuclide names to MARIS data formats

Below, we map nuclide names used by HELCOM to the MARIS standard nuclide names. 

Remapping data provider nomenclatures to MARIS standards is a recurrent operation and is done in a semi-automated manner according to the following pattern:

1. **Inspect** data provider nomenclature:
2. **Match** automatically against MARIS nomenclature (using a fuzzy matching algorithm); 
3. **Fix** potential mismatches; 
4. **Apply** the lookup table to the dataframe.

We will refer to this process as **IMFA** (**I**nspect, **M**atch, **F**ix, **A**pply).

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: The `Nuclide` column has inconsistent naming. E.g:

- `Cs-137`,  `137Cs` or `CS-137`
- `239, 240 pu` or `239,240 pu`
- `ra-226` and `226ra` 

See below:

:::

In [ ]:
#| eval: false
get_unique_across_dfs(dfs_output, col_name='Nuclide', as_df=True)

,index,value
0,0,228ra
1,1,NaN
2,2,"239, 240 pu"
3,3,3h
4,4,238pu
5,5,"239,240pu"
6,6,99tc
7,7,226ra
8,8,137cs
9,9,cs-137


Let's now create an instance of a [fuzzy matching algorithm](https://www.wikiwand.com/en/articles/Approximate_string_matching) `Remapper`. This instance will match the nuclide names of the OSPAR dataset to the MARIS standard nuclide names.

In [ ]:
#| eval: false
remapper = Remapper(provider_lut_df=get_unique_across_dfs(dfs_output, col_name='Nuclide', as_df=True),
                    maris_lut_fn=nuc_lut_path,
                    maris_col_id='nuclide_id',
                    maris_col_name='nc_name',
                    provider_col_to_match='value',
                    provider_col_key='value',
                    fname_cache='nuclides_ospar.pkl')

Lets try to match OSPAR nuclide names to MARIS standard nuclide names as automatically as possible. The `match_score` column allows to assess the results:

In [ ]:
#| eval: false
remapper.generate_lookup_table(as_df=True)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:  38%|███▊      | 5/13 [00:00<00:00, 39.26it/s]

Processing: 100%|██████████| 13/13 [00:00<00:00, 34.88it/s]

1 entries matched the criteria, while 12 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
"239, 240 pu",pu240,"239, 240 pu",8
"239,240pu",pu240,"239,240pu",6
228ra,u235,228ra,4
226ra,u234,226ra,4
137cs,i133,137cs,4
210po,ru106,210po,4
210pb,ru106,210pb,4
241am,pu241,241am,4
238pu,u238,238pu,3


We can now manually inspect the unmatched nuclide names and create a table to correct them to the MARIS standard:

In [ ]:
#| exports
fixes_nuclide_names = {
    '99tc': 'tc99',
    '238pu': 'pu238',
    '226ra': 'ra226',
    '210pb': 'pb210',
    '241am': 'am241',
    '228ra': 'ra228',
    '137cs': 'cs137',
    '210po': 'po210',
    '239,240pu': 'pu239_240_tot',
    '239, 240 pu': 'pu239_240_tot',
    'cs-137': 'cs137',
    '3h': 'h3'
    }

We now include the table `fixes_nuclide_names`, which applies manual corrections to the nuclide names before the remapping process. 
The `generate_lookup_table` function has an `overwrite` parameter (default is `True`), which, when set to `True`, creates a pickle file cache of the lookup table. We can now test the remapping process:

In [ ]:
#| eval: false
remapper.generate_lookup_table(as_df=True, fixes=fixes_nuclide_names)
fc.test_eq(len(remapper.select_match(match_score_threshold=1)), 0)

Processing:   0%|          | 0/13 [00:00<?, ?it/s]

Processing: 100%|██████████| 13/13 [00:00<00:00, 36.78it/s]


If we want to view all the remapped nuclides we can set the match score threshold to 0; 

In [ ]:
#| eval: false
remapper.generate_lookup_table(as_df=True, fixes=fixes_nuclide_names)
remapper.select_match(match_score_threshold=0, verbose=True)

Processing:   0%|          | 0/13 [00:00<?, ?it/s]

Processing: 100%|██████████| 13/13 [00:00<00:00, 28.98it/s]

0 entries matched the criteria, while 13 entries had a match score of 0 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
228ra,ra228,228ra,0
NaN,Unknown,NaN,0
"239, 240 pu",pu239_240_tot,"239, 240 pu",0
3h,h3,3h,0
238pu,pu238,238pu,0
"239,240pu",pu239_240_tot,"239,240pu",0
99tc,tc99,99tc,0
226ra,ra226,226ra,0
137cs,cs137,137cs,0


Values are remapped correctly! We can now create a callback `RemapNuclideNameCB` to remap the nuclide names. Note that we pass `overwrite=False` to the `Remapper` constructor to now use the cached version.

In [ ]:
#| exports
# Create a lookup table for nuclide names
lut_nuclides = lambda df: Remapper(provider_lut_df=df,
                                   maris_lut_fn=nuc_lut_path,
                                   maris_col_id='nuclide_id',
                                   maris_col_name='nc_name',
                                   provider_col_to_match='value',
                                   provider_col_key='value',
                                   fname_cache='nuclides_ospar.pkl').generate_lookup_table(fixes=fixes_nuclide_names, 
                                                                                            as_df=False, overwrite=False)

In [ ]:
#| exports
class RemapNuclideNameCB(Callback):
    "Remap data provider nuclide names to standardized MARIS nuclide names."
    def __init__(self, 
                 fn_lut: Callable, # Function that returns the lookup table dictionary
                 col_name: str # Column name to remap
                ):
        fc.store_attr()

    def __call__(self, tfm: Transformer):
        df_uniques = get_unique_across_dfs(tfm.dfs, col_name=self.col_name, as_df=True)
        #lut = {k: v.matched_maris_name for k, v in self.fn_lut(df_uniques).items()}    
        lut = {k: v.matched_id for k, v in self.fn_lut(df_uniques).items()}    
        for k in tfm.dfs.keys():
            tfm.dfs[k]['NUCLIDE'] = tfm.dfs[k][self.col_name].replace(lut)

Let's see it in action, along with the `LowerStripNameCB` callback:

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            LowerStripNameCB(col_src='Nuclide', col_dst='Nuclide'),
                            RemapNuclideNameCB(lut_nuclides, col_name='Nuclide')
                            ])
dfs_out = tfm()

# For instance
for key in dfs_out.keys():
    print(f'{key} NUCLIDE unique: ', dfs_out[key]['NUCLIDE'].unique())

SEAWATER NUCLIDE unique:  [33 77 53 54 15  1 47 41 -1]
BIOTA NUCLIDE unique:  [33 53 54 77 15 47 41  1 67 72]


## Standardize Time

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: `SEAWATER` dataset contains 1O rows with `NaN` values in the `Sampling date` column as shown below.

:::

In [ ]:
#| eval: false
dfs = load_data(fname_in)
print('Number of NaN values in Sampling date: ', dfs['SEAWATER']['Sampling date'].isnull().sum())


Number of NaN values in Sampling date:  10


Create a callback that remaps the time format in the dictionary of dataframes (i.e. `%m/%d/%y %H:%M:%S`) and handle missing dates:

In [ ]:
#| exports
class ParseTimeCB(Callback):
    "Parse the time format in the dataframe."
    def __call__(self, tfm):
        for df in tfm.dfs.values():
            df['TIME'] = pd.to_datetime(df['Sampling date'], format='%m/%d/%y %H:%M:%S', errors='coerce')
            df.dropna(subset=['TIME'], inplace=True)

Apply the transformer for callbacks `ParseTimeCB`. Then, print the ``begperiod`` and `time` data for `seawater`.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
    ParseTimeCB(),
    CompareDfsAndTfmCB(dfs)])

tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['SEAWATER']['TIME'])

                           SEAWATER  BIOTA
Number of rows in dfs         19193  15951
Number of rows in tfm.dfs     19183  15951
Number of rows removed           10      0 

0       2010-01-27 00:00:00
1       2010-01-27 00:00:00
2       2010-01-27 00:00:00
3       2010-01-27 00:00:00
4       2010-01-26 00:00:00
                ...        
19183   2019-11-13 12:54:00
19184   2019-12-10 11:37:00
19185   2019-12-10 11:37:00
19186   2019-12-10 11:37:00
19187   2019-12-18 14:43:00
Name: TIME, Length: 19183, dtype: datetime64[ns]


The NetCDF time format requires the time to be encoded as number of milliseconds since a time of origin. In our case the time of origin is `1970-01-01` as indicated in `configs.ipynb` `CONFIFS['units']['time']` dictionary.

`EncodeTimeCB` converts the HELCOM `time` format to the MARIS NetCDF `time` format.

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[ParseTimeCB(),
                            EncodeTimeCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.logs)
                            

                           SEAWATER  BIOTA
Number of rows in dfs         19193  15951
Number of rows in tfm.dfs     19183  15951
Number of rows removed           10      0 

['Parse the time format in the dataframe.', 'Encode time as seconds since epoch.', 'Create a dataframe of dropped data. Data included in the `dfs` not in the `tfm`.']


## Sanitize value

We allocate each column containing measurement values into a single column `VALUE` and remove `NA` where needed.

In [ ]:
# | exports
class SanitizeValueCB(Callback):
    "Sanitize value by removing blank entries and populating `value` column."
    def __init__(self, 
                 value_col: str='Activity or MDA' # Column name to sanitize
                 ):
        fc.store_attr()

    def __call__(self, tfm):
        for df in tfm.dfs.values():
            df.dropna(subset=[self.value_col], inplace=True)
            df['VALUE'] = df[self.value_col]

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[SanitizeValueCB(),
                            CompareDfsAndTfmCB(dfs)])

tfm()

print('Example of VALUE column:')
print(tfm.dfs['SEAWATER'][['VALUE']].head())
print('\nComparison stats:')
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

Example of VALUE column:
   VALUE
0   0.20
1   0.27
2   0.26
3   0.25
4   0.20

Comparison stats:
                           SEAWATER  BIOTA
Number of rows in dfs         19193  15951
Number of rows in tfm.dfs     19183  15951
Number of rows removed           10      0 



## Normalize uncertainty

For each sample type in the OSPAR dataset, the reported uncertainty is given as an expanded uncertainty with a coverage factor `𝑘=2`. For further details, refer to the [OSPAR reporting guidelines](https://mcc.jrc.ec.europa.eu/documents/OSPAR/Guidelines_forestimationof_a_%20measurefor_uncertainty_in_OSPARmonitoring.pdf).

**Note**: For MARIS the OSPAR uncertainty values are normalized to standard uncertainty with a coverage factor 
𝑘=1.

`NormalizeUncCB` callback normalizes the uncertainty using the following `lambda` function:

In [ ]:
#| exports
unc_exp2stan = lambda df, unc_col: df[unc_col] / 2

In [ ]:
#| exports
class NormalizeUncCB(Callback):
    """Normalize uncertainty values in DataFrames."""
    def __init__(self, 
                 col_unc: str='Uncertainty', # Column name to normalize
                 fn_convert_unc: Callable=unc_exp2stan, # Function correcting coverage factor
                 ): 
        fc.store_attr()

    def __call__(self, tfm):
        for df in tfm.dfs.values():
            df['UNCERTAINTY'] = self.fn_convert_unc(df, self.col_unc)

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
        SanitizeValueCB(),               
        NormalizeUncCB()
    ])
tfm()

for grp in ['SEAWATER', 'BIOTA']:
    print(f'\n{grp}:')
    print(tfm.dfs[grp][['VALUE', 'UNCERTAINTY']].head())


SEAWATER:
   VALUE  UNCERTAINTY
0   0.20          NaN
1   0.27          NaN
2   0.26          NaN
3   0.25          NaN
4   0.20          NaN

BIOTA:
      VALUE  UNCERTAINTY
0  0.326416          NaN
1  0.442704          NaN
2  0.412989          NaN
3  0.202768          NaN
4  0.652833          NaN


## Remap units

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: It would be easier to work with the units if they were standardized. The units are not consistent across the dataset, for instance `BQ/L`, `Bq/l` and `Bq/L` are used interchangeably.

:::


:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: The `Unit` column contains `NaN` values for the `SEAWATER` dataset, as shown below.
:::


In [ ]:
dfs['SEAWATER'].columns

Index(['ID', 'Contracting Party', 'RSC Sub-division', 'Station ID',
       'Sample ID', 'LatD', 'LatM', 'LatS', 'LatDir', 'LongD', 'LongM',
       'LongS', 'LongDir', 'Sample type', 'Sampling depth', 'Sampling date',
       'Nuclide', 'Value type', 'Activity or MDA', 'Uncertainty', 'Unit',
       'Data provider', 'Measurement Comment', 'Sample Comment',
       'Reference Comment'],
      dtype='object')

In [ ]:
dfs['SEAWATER'][dfs['SEAWATER']['Unit'].isnull()].drop(columns=['Measurement Comment','Sample Comment','Reference Comment']).head()

,ID,Contracting Party,RSC Sub-division,Station ID,Sample ID,LatD,LatM,LatS,LatDir,LongD,...,LongDir,Sample type,Sampling depth,Sampling date,Nuclide,Value type,Activity or MDA,Uncertainty,Unit,Data provider
16161,120369,Ireland,1.0,Salthill,NaN,53,15.0,40.0,N,9,...,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16162,120370,Ireland,1.0,Woodstown,NaN,52,11.0,55.0,N,6,...,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16586,120363,Ireland,4.0,N1,NaN,53,25.0,0.0,N,6,...,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19188,120364,Ireland,4.0,N2,NaN,53,36.0,0.0,N,5,...,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19189,120365,Ireland,4.0,N3,NaN,53,44.0,0.0,N,5,...,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's inspect the unique units used by OSPAR:

In [ ]:
get_unique_across_dfs(dfs, col_name='Unit', as_df=True)

,index,value
0,0,NaN
1,1,Bq/l
2,2,Bq/L
3,3,Bq/kg f.w.
4,4,BQ/L


We define unit renaming rules for OSPAR dataset:

In [ ]:
#| export
# Define unit names renaming rules
renaming_unit_rules = {'Bq/l': 1, #'Bq/m3'
                       'Bq/L': 1,
                       'BQ/L': 1,
                       'Bq/kg f.w.': 5, # Bq/kgw
                       } 

In [ ]:
#| exports
class RemapUnitCB(Callback):
    """Callback to update DataFrame 'UNIT' columns based on a lookup table."""

    def __init__(self, lut: Dict[str, str]):
        fc.store_attr('lut')  # Store the lookup table as an attribute

    def __call__(self, tfm: 'Transformer'):
        for grp, df in tfm.dfs.items():
            #if grp == 'seawater':
            #    self._apply_default_units(df)
            self._print_na_units(df)
            self._update_units(df)

    def _apply_default_units(self, df: pd.DataFrame):
        df.loc[df['Unit'].isnull(), 'Unit'] = 'Bq/l'

    def _print_na_units(self, df: pd.DataFrame):
        na_count = df['Unit'].isnull().sum()
        if na_count > 0:
            print(f"Number of rows with NaN in 'Unit' column: {na_count}")

    def _update_units(self, df: pd.DataFrame):
        df['UNIT'] = df['Unit'].apply(lambda x: self.lut.get(x, 'Unknown'))

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[SanitizeValueCB(), # Remove blank value entries (also removes NaN values in Unit column) 
                            RemapUnitCB(renaming_unit_rules),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print('Unit unique values:')
for grp in ['BIOTA', 'SEAWATER']:
    print(f"{grp}: {tfm.dfs[grp]['UNIT'].unique()}")

                           SEAWATER  BIOTA
Number of rows in dfs         19193  15951
Number of rows in tfm.dfs     19183  15951
Number of rows removed           10      0 

Unit unique values:
BIOTA: [5]
SEAWATER: [1]


## Remap detection limit

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: The `Value type` column contains many `nan` values, see below.
:::

In [ ]:
# Count the number of NaN entries in the 'Value type' column for 'SEAWATER'
na_count_seawater = dfs['SEAWATER']['Value type'].isnull().sum()
print(f"Number of NaN 'Value type' entries in 'SEAWATER': {na_count_seawater}")

# Count the number of NaN entries in the 'Value type' column for 'BIOTA'
na_count_biota = dfs['BIOTA']['Value type'].isnull().sum()
print(f"Number of NaN 'Value type' entries in 'BIOTA': {na_count_biota}")

Number of NaN 'Value type' entries in 'SEAWATER': 64
Number of NaN 'Value type' entries in 'BIOTA': 23


In the OSPAR dataset the detection limit is encoded as `<`  in the `Value type` column. If a value is `<` then the `Activity or MDA` column contains the detection limit value. If the `Value type` is `=` then the `Activity or MDA` column contains the measurement value.


Lets review the `Value type` column values for the OSPAR dataset:

In [ ]:
for grp in dfs.keys():
    print(f'{grp}:')
    print(tfm.dfs[grp]['Value type'].unique())


SEAWATER:
['<' '=' nan]
BIOTA:
['<' '=' nan]


Detection limits are encoded as follows in MARIS:

In [ ]:
#| eval: false
pd.read_excel(detection_limit_lut_path())

,id,name,name_sanitized
0,-1,Not applicable,Not applicable
1,0,Not Available,Not available
2,1,=,Detected value
3,2,<,Detection limit
4,3,ND,Not detected
5,4,DE,Derived


In [ ]:
#| exports
lut_dl = lambda: pd.read_excel(detection_limit_lut_path(), usecols=['name','id']).set_index('name').to_dict()['id']

In [ ]:
#| exports
coi_dl = {'SEAWATER' : {'DL' : 'Value type'},
          'BIOTA':  {'DL' : 'Value type'}
          }

In [ ]:
# | exports
class RemapDetectionLimitCB(Callback):
    "Remap value type to MARIS format."
    
    def __init__(self, 
                 coi: dict,  # Column configuration dictionary
                 fn_lut: Callable  # Lookup table dictionary
                ):
        fc.store_attr()

    def __call__(self, tfm: Transformer):
        
        lut = self.fn_lut()
        
        "Remap detection limits in the DataFrames using the lookup table."
        for grp in tfm.dfs:
            df = tfm.dfs[grp]
            self._update_detection_limit(df, grp, lut)

    def _update_detection_limit(self, 
                                df: pd.DataFrame,  # The DataFrame to modify
                                grp: str,  # The group name to get the column configuration
                                lut: dict  # The lookup table dictionary
                               ) -> None:
        "Update detection limit column in the DataFrame based on lookup table and rules."
        
        print('Attempting to update detection limit column...')
        print(f'Group: {grp}')
        print(self.coi)
        
        # Access column names from coi_dl
        detection_col = self.coi[grp]['DL']   
        
        print(f'Detection column: {detection_col}') 
        
        # Initialize detection limit column
        df['DL'] = df[detection_col]
        
        # Set detection limits based on conditions
        self._set_detection_limits(df, lut)

    def _set_detection_limits(self, df: pd.DataFrame, lut: dict) -> None:
        "Set detection limits based on value and uncertainty columns."
        
        # Condition for setting '='
        condition_eq = (df['VALUE'].notna() & 
                        df['UNCERTAINTY'].notna() & 
                        ~df['DL'].isin(lut.keys()))
        
        df.loc[condition_eq, 'DL'] = '='

        # Set 'Not Available' for unmatched detection limits
        df.loc[~df['DL'].isin(lut.keys()), 'DL'] = 'Not Available'
        
        # Perform lookup to map detection limits
        df['DL'] = df['DL'].map(lut)

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[SanitizeValueCB(),
                            NormalizeUncCB(),                  
                            RemapUnitCB(renaming_unit_rules),
                            RemapDetectionLimitCB(coi_dl, lut_dl)])
tfm()
for grp in ['BIOTA', 'SEAWATER']:
    print(f"{grp}: {tfm.dfs[grp]['DL'].unique()}")

Attempting to update detection limit column...
Group: SEAWATER
{'SEAWATER': {'DL': 'Value type'}, 'BIOTA': {'DL': 'Value type'}}
Detection column: Value type
Attempting to update detection limit column...
Group: BIOTA
{'SEAWATER': {'DL': 'Value type'}, 'BIOTA': {'DL': 'Value type'}}
Detection column: Value type
BIOTA: [2 1]
SEAWATER: [2 1]


## Remap Biota species

We will remap the OSPAR `Species` column to the MARIS `SPECIES` column using the **IMFA** (**I**nspect, **M**atch, **F**ix, **A**pply) pattern. First lets **inspect** inspect unique `Species` values used by OSPAR:

In [ ]:
dfs = load_data(fname_in)
get_unique_across_dfs(dfs, col_name='Species', as_df=True)

,index,value
0,0,FUCUS SPP.
1,1,NaN
2,2,LITTORINA LITTOREA
3,3,Melanogrammus aeglefinus
4,4,MELANOGRAMMUS AEGLEFINUS
...,...,...
162,162,Gadus sp.
163,163,Pleuronectes platessa
164,164,Gadiculus argenteus
165,165,Cyclopterus lumpus


Now we try to **MATCH** the `Species` column of ``OSPAR`` ``BIOTA`` dataset to the `species` column of the MARIS species lookup table, again using a `Remapper` object:

In [ ]:
#| eval: false
remapper = Remapper(provider_lut_df=get_unique_across_dfs(dfs, col_name='Species', as_df=True),
                    maris_lut_fn=species_lut_path,
                    maris_col_id='species_id',
                    maris_col_name='species',
                    provider_col_to_match='value',
                    provider_col_key='value',
                    fname_cache='species_ospar.pkl')

In [ ]:
remapper.generate_lookup_table(as_df=True)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/167 [00:00<?, ?it/s]

Processing:  80%|███████▉  | 133/167 [00:33<01:11,  2.10s/it]

Below, we will correct the entries that were not properly matched by the `Remapper` object:

In [ ]:
#|exports
fixes_biota_species = {
    'RHODYMENIA PSEUDOPALAMATA & PALMARIA PALMATA': NA,    
    'Mixture of green, red and brown algae': NA,
    'SOLEA SOLEA (S.VULGARIS)': 'Solea solea',
    'Solea solea (S.vulgaris)': 'Solea solea',
    'RAJIDAE/BATOIDEA': NA,
    'PALMARIA PALMATA': NA,
    'Gadiculus argenteus': 'Gadiculus argenteus thori',
    'MONODONTA LINEATA': 'Phorcus lineatus', #PMG please check is this is correct
    'Unknown': NA,
    'unknown': NA,
    'Flatfish': 'Pisces',
    'Gadus sp.': 'Gadus'
    }

And give it an another try:

In [ ]:
#| eval: false
remapper.generate_lookup_table(fixes=fixes_biota_species)
remapper.select_match(match_score_threshold=1)

Processing:   0%|          | 0/167 [00:00<?, ?it/s]

Processing: 100%|██████████| 167/167 [00:32<00:00,  5.20it/s]


,matched_maris_name,source_name,match_score
source_key,,,
CERASTODERMA (CARDIUM) EDULE,Cerastoderma edule,CERASTODERMA (CARDIUM) EDULE,10
Cerastoderma (Cardium) Edule,Cerastoderma edule,Cerastoderma (Cardium) Edule,10
DICENTRARCHUS (MORONE) LABRAX,Dicentrarchus labrax,DICENTRARCHUS (MORONE) LABRAX,9
Pleuronectiformes [order],Pleuronectiformes,Pleuronectiformes [order],8
FUCUS SPP.,Fucus,FUCUS SPP.,5
Rhodymenia spp.,Rhodymenia,Rhodymenia spp.,5
Sepia spp.,Sepia,Sepia spp.,5
RAJA DIPTURUS BATIS,Dipturus batis,RAJA DIPTURUS BATIS,5
Thunnus sp.,Thunnus,Thunnus sp.,4


Visual inspection of the remaining unperfectly matched entries seem acceptable to proceed. 

We can now use the generic `RemapCB` callback to perform the remapping of the `Species` column to the `SPECIES` column after having defined the lookup table `lut_biota`.

In [ ]:
#| exports
lut_biota = lambda: Remapper(provider_lut_df=get_unique_across_dfs(dfs, col_name='Species', as_df=True),
                             maris_lut_fn=species_lut_path,
                             maris_col_id='species_id',
                             maris_col_name='species',
                             provider_col_to_match='value',
                             provider_col_key='value',
                             fname_cache='species_ospar.pkl'
                             ).generate_lookup_table(fixes=fixes_biota_species, as_df=False, overwrite=False)

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
    RemapCB(fn_lut=lut_biota, col_remap='SPECIES', col_src='Species', dest_grps='BIOTA', verbose=True)
    ])
tfm()
tfm.dfs['BIOTA'].columns
# For instance:
print(tfm.dfs['BIOTA']['SPECIES'].unique())

Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched value: nan
Unmatched val

## Remap Biota species enhanced

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Many entries for `Species` in the Ospar biota dataset are nan, see below.
:::

In [ ]:
dfs['BIOTA']['Species'].isna().sum()

2198

The values that include a `-1` in the `SPECIES` column are not matched by the `Remapper` object. The corresponding `Biological group` values are:

In [ ]:
dfs['BIOTA'][tfm.dfs['BIOTA']['SPECIES'] == -1]['Biological group'].unique()

array(['Fish'], dtype=object)

The ``Biological group`` column in the ``OSPAR`` dataset provides valuable insights related to ``SPECIES``. We will leverage this information to enrich the ``species`` column. To achieve this, we will employ the generic ``RemapCB`` callback to create an ``enhanced_species`` column. Subsequently, this enhanced_species column will be used to further enrich the species column.

First we inspect the unique values in the Biological group column.

In [ ]:
get_unique_across_dfs(dfs, col_name='Biological group', as_df=True)

,index,value
0,0,Seaweeds
1,1,Molluscs
2,2,molluscs
3,3,fish
4,4,Fish
5,5,Seaweed
6,6,SEAWEED
7,7,FISH
8,8,seaweed
9,9,MOLLUSCS


We will remap the Biological group columns data to the species column of the MARIS nomenclature, again using a Remapper object:

In [ ]:
#| eval: false
remapper = Remapper(provider_lut_df=get_unique_across_dfs(dfs, col_name='Biological group', as_df=True),
                    maris_lut_fn=species_lut_path,
                    maris_col_id='species_id',
                    maris_col_name='species',
                    provider_col_to_match='value',
                    provider_col_key='value',
                    fname_cache='enhance_species_ospar.pkl')

Like before we will generate the lookup table and select matches that meet a specified threshold (i.e., greater than 1), which means that matches requiring more than one character change are shown.



In [ ]:
remapper.generate_lookup_table(as_df=True)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/10 [00:00<?, ?it/s]

Processing: 100%|██████████| 10/10 [00:02<00:00,  3.79it/s]

3 entries matched the criteria, while 7 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
fish,Fucus,fish,4
Fish,Fucus,Fish,4
FISH,Fucus,FISH,4
Seaweeds,Seaweed,Seaweeds,1
Molluscs,Mollusca,Molluscs,1
molluscs,Mollusca,molluscs,1
MOLLUSCS,Mollusca,MOLLUSCS,1


`fixes_biota_species_enhanced` applies manual fixes.

In [ ]:
#|exports
fixes_enhanced_biota_species = {
    'fish': 'Pisces',
    'FISH': 'Pisces',
    'Fish': 'Pisces'    
    
}

Now we will apply the manual corrections to the lookup table and generate the lookup table again.



In [ ]:
#| eval: false
remapper.generate_lookup_table(fixes=fixes_enhanced_biota_species)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/10 [00:00<?, ?it/s]

Processing: 100%|██████████| 10/10 [00:02<00:00,  4.54it/s]

6 entries matched the criteria, while 4 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
Seaweeds,Seaweed,Seaweeds,1
Molluscs,Mollusca,Molluscs,1
molluscs,Mollusca,molluscs,1
MOLLUSCS,Mollusca,MOLLUSCS,1


Visual inspection of the remaining imperfectly matched entries appears acceptable. We can now proceed with the final remapping process:

Create Remapper Lambda Function:

We'll define a lambda function that instantiates a Remapper object and returns its corrected lookup table.

Apply RemapCB:

Using the generic RemapCB callback, we'll perform the actual remapping.

In [ ]:

#| exports
lut_biota_enhanced = lambda: Remapper(provider_lut_df=get_unique_across_dfs(dfs, col_name='Biological group', as_df=True),
                             maris_lut_fn=species_lut_path,
                             maris_col_id='species_id',
                             maris_col_name='species',
                             provider_col_to_match='value',
                             provider_col_key='value',
                             fname_cache='enhance_species_ospar.pkl').generate_lookup_table(fixes=fixes_enhanced_biota_species, as_df=False, overwrite=False)

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
    RemapCB(fn_lut=lut_biota_enhanced, col_remap='enhanced_species', col_src='Biological group', dest_grps='BIOTA')    
    ])

tfm()['BIOTA']['enhanced_species'].unique()

array([ 873, 1059,  712])

Now that we have the enhanced_species column, we can use it to enrich the species column. We will use the enhanced species column in the absence of a species match if the enhanced species column is valid.

In [ ]:
# | export
class EnhanceSpeciesCB(Callback):
    """Enhance the 'species' column using the 'enhanced_species' column if conditions are met."""

    def __init__(self):
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        self._enhance_species(tfm.dfs['BIOTA'])

    def _enhance_species(self, df: pd.DataFrame):
        df['SPECIES'] = df.apply(
            lambda row: row['enhanced_species'] if row['SPECIES'] in [-1, 0] and pd.notnull(row['enhanced_species']) else row['SPECIES'],
            axis=1
        )

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
    RemapCB(fn_lut=lut_biota, col_remap='SPECIES', col_src='Species', dest_grps='BIOTA'),
    RemapCB(fn_lut=lut_biota_enhanced, col_remap='enhanced_species', col_src='Biological group', dest_grps='BIOTA'),
    EnhanceSpeciesCB()
    ])

tfm()['BIOTA']['SPECIES'].unique()

array([ 377,  129,   96,  712,  192,   99,   50,  378,  270,  379,  380,
        381,  382,  383,  384,  385,  244,  386,  387,  388,  389,  390,
        391,  392,  393,  394,  395,  396,  274,  397,  398,  243,  399,
        400,  401,  402,  403,  404,  405,  406,  407, 1059,  191,  139,
        408,  409,  410,  412,  413,  272,  414,  415,  416,  417,  418,
        419,  420,  421,  422,  423,  424,  425,  426,  427,  428,  411,
        429,  430,  431,  432,  433,  434,  435,  436,  437,  438,  439,
        440,  441,  442,  443,  444,  294, 1607, 1610, 1609, 1605, 1608,
         23, 1606,  234,  556, 1701, 1752,  158])

## Remap Biota tissues

:::{.callout-tip}

FEEDBACK TO DATA PROVIDER: biota dataset includes 1 entry where the Body Part is FLESH WITHOUT BONES for the Biological group of SEAWEED, see below.

:::

In [ ]:
dfs['BIOTA'][['ID','Contracting Party','Sample ID','Biological group','Body Part', 'Measurement Comment', 'Sample Comment']][(tfm.dfs['BIOTA']['Body Part'] == 'FLESH WITHOUT BONES') & (tfm.dfs['BIOTA']['Biological group'] == 'SEAWEED')]


,ID,Contracting Party,Sample ID,Biological group,Body Part,Measurement Comment,Sample Comment
13057,87356,Iceland,THFAG17C,SEAWEED,FLESH WITHOUT BONES,NaN,NaN


The ``OSPAR`` dataset includes entries where the ``Body Part`` is labeled as ``whole``. However, the MARIS data standard requires a more specific distinction in the ``body_part`` field, differentiating between ``Whole animal`` and ``Whole plant``. Fortunately, the OSPAR data provides a Biological group field that allows us to make this distinction.

To address this discrepancy and ensure compatibility with MARIS standards, we will:

1. Create a temporary column body_part_temp that combines information from both Body Part and Biological group.
2. Use this temporary column to perform the lookup using our Remapper object.

Lets create the temporary column, body_part_temp, that combines Body Part and Biological group.

In [ ]:
#| exports
class AddBodyPartTempCB(Callback):
    "Add a temporary column with the body part and biological group combined."    
    def __call__(self, tfm):
        tfm.dfs['BIOTA']['body_part_temp'] = (
            tfm.dfs['BIOTA']['Body Part'] + ' ' + 
            tfm.dfs['BIOTA']['Biological group']
            ).astype(str).str.strip().str.lower()

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[  
                            AddBodyPartTempCB(),
                            ])
dfs_test = tfm()
dfs_test['BIOTA']['body_part_temp'].unique()

array(['whole animal molluscs', 'whole plant seaweed', 'whole fish fish',
       'flesh without bones fish', 'whole animal fish', 'muscle fish',
       'head fish', 'soft parts molluscs', 'growing tips seaweed',
       'soft parts fish', 'unknown fish', 'flesh without bone fish',
       'flesh fish', 'flesh with scales fish', 'liver fish',
       'flesh without bones seaweed', 'whole  fish',
       'flesh without bones molluscs', 'whole  seaweed',
       'whole plant seaweeds', 'whole fish', 'whole without head fish',
       'mix of muscle and whole fish without liver fish',
       'whole fisk fish', 'muscle  fish', 'cod medallion fish',
       'tail and claws fish'], dtype=object)

To align the body_part_temp column with the bodypar column in the MARIS nomenclature, we utilize a Remapper object. Since the OSPAR dataset does not include a predefined lookup table for the body_part column, we first create a lookup table by extracting unique values from the body_part_temp column.

In [ ]:
get_unique_across_dfs(dfs_test, col_name='body_part_temp', as_df=True).head(10)

,index,value
0,0,head fish
1,1,whole animal molluscs
2,2,growing tips seaweed
3,3,muscle fish
4,4,whole without head fish
5,5,whole fish fish
6,6,flesh without bones molluscs
7,7,unknown fish
8,8,whole plant seaweed
9,9,whole fish


In [ ]:
#| eval: false
remapper = Remapper(provider_lut_df=get_unique_across_dfs(dfs_test, col_name='body_part_temp', as_df=True),
                    maris_lut_fn=bodyparts_lut_path,
                    maris_col_id='bodypar_id',
                    maris_col_name='bodypar',
                    provider_col_to_match='value',
                    provider_col_key='value',
                    fname_cache='bodyparts_ospar.pkl'
                    )

remapper.generate_lookup_table(as_df=True)
remapper.select_match(match_score_threshold=0).head(5)

Processing:   0%|          | 0/27 [00:00<?, ?it/s]

Processing: 100%|██████████| 27/27 [00:00<00:00, 84.68it/s]


,matched_maris_name,source_name,match_score
source_key,,,
mix of muscle and whole fish without liver fish,Flesh without bones,mix of muscle and whole fish without liver fish,31
cod medallion fish,Old leaf,cod medallion fish,13
whole without head fish,Flesh without bones,whole without head fish,13
tail and claws fish,Stomach and intestine,tail and claws fish,13
whole plant seaweeds,Whole plant,whole plant seaweeds,9


Many of the lookup entries are sufficient for our needs. However, for values that don't find a match, we can use the fixes_biota_bodyparts dictionary to apply manual corrections. First we will create the dictionary.

In [ ]:
#|exports
fixes_biota_tissues = {
                    'mix of muscle and whole fish without liver fish': NA,
                    'cod medallion fish': NA,
                    'whole without head fish': 'Whole animal eviscerated without head',
                    'tail and claws fish': NA,
                    'whole plant seaweeds': NA,
                    'soft parts molluscs': NA,
                    'whole animal molluscs': NA,
                    'whole fisk fish': NA,
                    'unknown fish': NA,
                    'flesh without bones seaweed': NA, # This isnt possible
                    'flesh fish': 'Flesh without bones'
                    }


Lets correct the unmatched entries:

In [ ]:
#| eval: false
remapper.generate_lookup_table(fixes=fixes_biota_tissues)
remapper.select_match(match_score_threshold=1)

Processing:   0%|          | 0/27 [00:00<?, ?it/s]

Processing: 100%|██████████| 27/27 [00:00<00:00, 83.35it/s]


,matched_maris_name,source_name,match_score
source_key,,,
whole fish fish,Whole animal,whole fish fish,9
flesh without bones molluscs,Flesh without bones,flesh without bones molluscs,9
growing tips seaweed,Growing tips,growing tips seaweed,8
whole plant seaweed,Whole plant,whole plant seaweed,8
whole seaweed,Whole plant,whole seaweed,7
muscle fish,Muscle,muscle fish,6
head fish,Head,head fish,5
muscle fish,Muscle,muscle fish,5
whole fish,Whole animal,whole fish,5


At this stage, the majority of entries have been successfully matched to MARIS nomenclature. For those entries that remain unmatched, they are appropriately marked as not available. We can now proceed with the final remapping process:

- Create Remapper Lambda Function:

    We'll define a lambda function that instantiates a Remapper object and returns its corrected lookup table.

- Apply RemapCB:

    Using the generic RemapCB callback, we'll perform the actual remapping.

In [ ]:
#| exports
lut_tissues = lambda: Remapper(provider_lut_df=get_unique_across_dfs(tfm.dfs, col_name='body_part_temp', as_df=True),
                               maris_lut_fn=bodyparts_lut_path,
                               maris_col_id='bodypar_id',
                               maris_col_name='bodypar',
                               provider_col_to_match='value',
                               provider_col_key='value',
                               fname_cache='bodyparts_ospar.pkl'
                               ).generate_lookup_table(fixes=fixes_biota_tissues, as_df=False, overwrite=False)

Putting it all together, we now apply the RemapCB to our data. This process results in the addition of a body_part column to our biota dataframe, containing standardized species IDs.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[  
                            AddBodyPartTempCB(),
                            RemapCB(fn_lut=lut_tissues, col_remap='BODY_PART', col_src='body_part_temp' , dest_grps='BIOTA')
                            ])
tfm()
tfm.dfs['BIOTA']['BODY_PART'].unique()

array([ 0, 40,  1, 52, 34, 13, 56, 19, 60, 25,  3])

In [ ]:
#|export
unmatched_fixes_biota_tissues = {
'Mix of muscle and whole fish without liver' : 'Not available', # Drop
 'Whole without head' : 'Whole animal eviscerated without head', # Drop? eviscerated? ,
 'Cod medallion' : 'Whole animal eviscerated without head',
 'FLESH' : 'Flesh without bones', # Drop? with or without bones?
 'Flesh' : 'Flesh without bones', # Drop? with or without bones?
 'UNKNOWN' : 'Not available',
 'FLESH WITHOUT BONE' : 'Flesh without bones'
}

In [ ]:
#|eval: false
# tissues_lut_df = get_maris_lut(df_biota=tfm.dfs['biota'], 
#                                 fname_cache='tissues_ospar.pkl', 
#                                 data_provider_name_col='body_part',
#                                 maris_lut=bodyparts_lut_path,
#                                 maris_id='bodypar_id',
#                                 maris_name='bodypar',
#                                 unmatched_fixes=unmatched_fixes_biota_tissues,
#                                 as_dataframe=True,
#                                 overwrite=True)
# tissues_lut_df

NameError: name 'get_maris_lut' is not defined

List unmatched OSPAR tissues:

In [ ]:
#|eval: false
tissues_lut_df[tissues_lut_df['match_score'] >= 1]['source_name'].tolist()

['Mix of muscle and whole fish without liver', 'UNKNOWN']

In [ ]:
#| export
class LookupBiotaBodyPartCB(Callback):
    """Update body part id based on MARIS dbo_bodypar.xlsx"""

    def __init__(self, fn_lut: Callable, unmatched_fixes_biota_tissues: Dict[str, str]):
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        lut = self.fn_lut(df_biota=tfm.dfs['biota'])
        self.drop_nan_species(tfm.dfs['biota'])
        self.drop_unmatched(tfm.dfs['biota'])
        self.perform_lookup(tfm.dfs['biota'], lut)

    def drop_nan_species(self, df: pd.DataFrame):
        """
        Drop rows where 'body_part' is NaN.

        Args:
            df (pd.DataFrame): The DataFrame to process.
        """
        df.dropna(subset=['body_part'], inplace=True)

    def drop_unmatched(self, df: pd.DataFrame):
        """
        Drop rows where the 'body_part' is in the unmatched_fixes_biota_tissues list with value 'Not available'.

        Args:
            df (pd.DataFrame): The DataFrame to process.
        """
        na_list = ['Not available']
        na_biota_tissues = [k for k, v in self.unmatched_fixes_biota_tissues.items() if v in na_list]
        df.drop(df[df['body_part'].isin(na_biota_tissues)].index, inplace=True)

    def perform_lookup(self, df: pd.DataFrame, lut: Dict[str, 'Match']):
        """
        Perform lookup to update 'body_part' with matched IDs.

        Args:
            df (pd.DataFrame): The DataFrame to process.
            lut (Dict[str, Match]): The lookup table.
        """
        df['body_part'] = df['body_part'].apply(lambda x: lut[x].matched_id if x in lut else x)


In [ ]:
#|eval: false
get_maris_bodypart=partial(get_maris_lut, 
                            fname_cache='tissues_ospar.pkl', 
                            data_provider_name_col='body_part',
                            maris_lut=bodyparts_lut_path,
                            maris_id='bodypar_id',
                            maris_name='bodypar',
                            unmatched_fixes=unmatched_fixes_biota_tissues,
                            as_dataframe=False,
                            overwrite=False)


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                            CorrectWholeBodyPartCB(),
                            LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['biota'][['Body Part', 'body_part']][:5])

                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15308
Number of dropped rows                                     0      6
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 

      Body Part  body_part
0    SOFT PARTS         19
1  GROWING TIPS         56
2    SOFT PARTS         19
3    SOFT PARTS         19
4  GROWING TIPS         56


#### Lookup : Biogroup

In [ ]:
# TO BE DONE
# Species column contains nan
# Niall replace with Biological group where missing
# In our case, we will use remapped species, once 
# done we can use internal MARIS lookup to remap to biota group. 
# If species is missing, we can use the biological group to perform the lookup. 

In [ ]:
# unmatched_fixes_biota_species.update({
# # Biological group corrections
# 'Molluscs' : 'Mollusca',
# 'Seaweed' : 'Seaweed',
# 'Fish' : 'Pisces',
# 'FISH' : 'Pisces',
# 'seaweed' : 'Seaweed',
# 'SEAWEED' : 'Seaweed',
# 'molluscs' : 'Mollusca',
# 'fish' : 'Pisces',
# 'MOLLUSCS' : 'Mollusca' })

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variable: ``bio_group``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: Biogroup is not included.*

`get_biogroup_lut` reads the file at `species_lut_path()` and from the contents of this file creates a dictionary linking `species_id` to `biogroup_id`.

In [ ]:
#| export
def get_biogroup_lut(maris_lut: str) -> dict:
    """
    Retrieve a lookup table for biogroup ids from a MARIS lookup table.

    Args:
        maris_lut (str): Path to the MARIS lookup table (Excel file).

    Returns:
        dict: A dictionary mapping species_id to biogroup_id.
    """
    species = pd.read_excel(maris_lut)
    return species[['species_id', 'biogroup_id']].set_index('species_id').to_dict()['biogroup_id']


`LookupBiogroupCB` applies the corrected `biota` `bio group` data obtained from the `get_maris_lut` function to the `biota` dataframe in the dictionary of dataframes, `dfs`.

In [ ]:
#| export
class LookupBiogroupCB(Callback):
    """Update biogroup id based on MARIS dbo_species.xlsx."""

    def __init__(self, fn_lut: Callable):
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        lut = self.fn_lut()
        self.update_bio_group(tfm.dfs['biota'], lut)

    def update_bio_group(self, df: pd.DataFrame, lut: dict):
        """
        Update the 'bio_group' column in the DataFrame based on the lookup table.

        Args:
            df (pd.DataFrame): The DataFrame to process.
            lut (Dict[str, Any]): The lookup table for updating 'bio_group'.
        """
        df['bio_group'] = df['species'].apply(lambda x: lut.get(x, -1))


Apply the transformer for callbacks ``LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species)``,``CorrectWholeBodyPartCB()``, ``LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues)``,             ``LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path()))``,   ``CompareDfsAndTfmCB(dfs)`` . Then, print the ``Body Part``, ``body_part``, ``species``,``bio_group`` for the `biota` dataframe.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                            CorrectWholeBodyPartCB(),
                            LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['biota'][['Body Part', 'body_part', 'species','bio_group']][:5])

                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15308
Number of dropped rows                                     0      6
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 

      Body Part  body_part  species  bio_group
0    SOFT PARTS         19      394         13
1  GROWING TIPS         56       96         11
2    SOFT PARTS         19      394         13
3    SOFT PARTS         19      394         13
4  GROWING TIPS         56       96         11


Biota data dropped due to 'unkown' Body Part.

In [ ]:
print(tfm.dfs_dropped['biota']['Body Part'].unique())

['Mix of muscle and whole fish without liver' 'UNKNOWN']


***

#### Lookup : Taxon Information

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variable: Not included`*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Taxonname`` , ``TaxonRepName``, ``Taxonrank``*

`get_taxonname_lut` reads the file at `species_lut_path()` and from the contents of this file creates a dictionary linking `species_id` to `Taxonname`.

In [ ]:
#| export
def get_taxon_info_lut(maris_lut: str) -> dict:
    """
    Retrieve a lookup table for Taxonname from a MARIS lookup table.

    Args:
        maris_lut (str): Path to the MARIS lookup table (Excel file).

    Returns:
        dict: A dictionary mapping species_id to biogroup_id.
    """
    species = pd.read_excel(maris_lut)
    return species[['species_id', 'Taxonname', 'Taxonrank','TaxonDB','TaxonDBID','TaxonDBURL']].set_index('species_id').to_dict()

# TODO include Commonname field after next MARIS data reconciling process.

In [ ]:

# | export
class LookupTaxonInformationCB(Callback):
    """Update taxon names based on MARIS species LUT (dbo_species.xlsx)."""
    def __init__(self, fn_lut: Callable[[], dict]):
        """
        Initialize the LookupTaxonNameCB with a function to generate the lookup table.

        Args:
            fn_lut (Callable[[], dict]): Function that returns the lookup table dictionary.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Update the 'taxon_name' column in the DataFrame using the lookup table and print unmatched species IDs.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        lut = self.fn_lut()
        
        
        self._set_taxon_rep_name(tfm.dfs['biota'])
        tfm.dfs['biota']['Taxonname'] =  tfm.dfs['biota']['species'].apply(lambda x: self._get_name_by_species_id(x, lut['Taxonname']))
        #df['Commonname'] = df['species'].apply(lambda x: self._get_name_by_species_id(x, lut['Commonname']))
        tfm.dfs['biota']['Taxonrank'] =  tfm.dfs['biota']['species'].apply(lambda x: self._get_name_by_species_id(x, lut['Taxonrank']))
        tfm.dfs['biota']['TaxonDB'] =  tfm.dfs['biota']['species'].apply(lambda x: self._get_name_by_species_id(x, lut['TaxonDB']))
        tfm.dfs['biota']['TaxonDBID'] =  tfm.dfs['biota']['species'].apply(lambda x: self._get_name_by_species_id(x, lut['TaxonDBID']))
        tfm.dfs['biota']['TaxonDBURL'] =  tfm.dfs['biota']['species'].apply(lambda x: self._get_name_by_species_id(x, lut['TaxonDBURL']))


    def _set_taxon_rep_name(self, df: pd.DataFrame):
        """
        Remap the 'TaxonRepName' column to the 'Species' column values.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
        """
        # Ensure both columns exist before attempting to remap
        if 'Species' in df.columns:
            df['TaxonRepName'] = df['Species']
        else:
            print("Warning: 'Species' column not found in DataFrame.")
            
            

    def _get_name_by_species_id(self, species_id: str, lut: dict) -> str:
        """
        Get the  name from the lookup table and print species ID if the taxon name is not found.

        Args:
            species_id (str): The species ID from the DataFrame.
            lut (dict): The lookup table dictionary.

        Returns:
            str: The name from the lookup table.
        """
        name = lut.get(species_id, 'Unknown')  # Default to 'Unknown' if not found
        if name == 'Unknown':
            print(f"Unmatched species ID: {species_id} for {lut.keys()[0]}")
        return name


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                            CorrectWholeBodyPartCB(),
                            LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupTaxonInformationCB(partial(get_taxon_info_lut, species_lut_path())),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['biota'][['Taxonname', 'Taxonrank','TaxonDB','TaxonDBID','TaxonDBURL']].drop_duplicates().head())

                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15308
Number of dropped rows                                     0      6
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 

               Taxonname Taxonrank   TaxonDB TaxonDBID  \
0     Littorina littorea   species  Wikidata    Q27935   
1      Fucus vesiculosus   species  Wikidata   Q754755   
15        Mytilus edulis   species  Wikidata    Q27855   
24       Clupea harengus   species  Wikidata  Q2396858   
28  Merlangius merlangus   species  Wikidata   Q273083   

                                TaxonDBURL  
0     https://www.wikidata.org/wiki/Q27935  
1    https://www.wikidata.org/wiki/Q754755  
15    https://www.wikidata.org/wiki/Q27855  
24  https://www.wikidata.org/wiki/Q2396858  
28   https://www.wikidata.org/wiki/Q273083  


***

#### Lookup : Units

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variable: ``unit``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Unit``.*

Create `renaming_unit_rules` to rename the units. 

In [ ]:
#| export
# Define unit names renaming rules
renaming_unit_rules = {'Bq/l': 1, #'Bq/m3'
                       'Bq/L': 1,
                       'BQ/L': 1,
                       'Bq/kg f.w.': 5, # Bq/kgw
                       'Bq/kg.fw' : 5,
                       'Bq/kg fw' : 5,
                       'Bq/kg f.w' : 5 
                       } 

In [ ]:
#| export
class LookupUnitCB(Callback):
    """Update the 'unit' column in DataFrames based on a lookup table.
    The class handles:
    - Assigning a default unit for NaN values in the 'Unit' column for specific groups.
    - Dropping rows with NaN values in the 'Unit' column.
    - Performing lookup to update the 'unit' column based on the provided lookup table."""

    def __init__(self, lut: dict = renaming_unit_rules):
        """
        Initialize the LookupUnitCB with a lookup table.

        Args:
            lut (dict): A dictionary used for lookup to update the 'unit' column.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Apply the callback to each DataFrame in the transformer.

        Args:
            tfm (Transformer): The transformer containing DataFrames to process.
        """
        for grp in tfm.dfs.keys():
            if grp == 'seawater':
                self._apply_units(tfm.dfs[grp])
            self._drop_na_units(tfm.dfs[grp])
            self._perform_lookup(tfm.dfs[grp])

    def _apply_units(self, df: pd.DataFrame):
        """
        Apply a default unit where the 'Unit' column is NaN.

        Args:
            df (pd.DataFrame): The DataFrame to process.
        """
        df.loc[df['Unit'].isnull(), 'Unit'] = 'Bq/l'

    def _drop_na_units(self, df: pd.DataFrame):
        """
        Drop rows where the 'Unit' column has NaN values.

        Args:
            df (pd.DataFrame): The DataFrame to process.
        """
        df.dropna(subset=['Unit'], inplace=True)

    def _perform_lookup(self, df: pd.DataFrame):
        """
        Perform lookup to update the 'unit' column based on the lookup table.

        Args:
            df (pd.DataFrame): The DataFrame to process.
        """
        df['unit'] = df['Unit'].apply(lambda x: self.lut.get(x, 'Unknown'))


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            LookupUnitCB(renaming_unit_rules),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['biota']['unit'].dtypes)

                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15314
Number of dropped rows                                     0      0
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 

int64


***

#### Lookup : Detection limit or Value type

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variable: ``detection_limit``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine foramt variable: ``Value type``.*

In [ ]:
#|eval: false
grp='biota'
tfm.dfs[grp]['Value type'].unique()

array(['=', '<', '>', nan], dtype=object)

In [ ]:
# | export
class LookupDetectionLimitCB(Callback):
    """Remap activity value, activity uncertainty, and detection limit to MARIS format.
    This class performs the following operations:
    - Reads a lookup table from an Excel file.
    - Copies and processes the 'Value type' column.
    - Fills NaN values with 'Not Available'.
    - Drops rows where 'Value type' is not in the lookup table.
    - Performs a lookup to update the 'detection_limit' column based on the lookup table.
    """

    def __init__(self, lut_path: str):
        """
        Initialize the LookupDetectionLimitCB with a path to the lookup table.

        Args:
            lut_path (str): The path to the Excel file containing the lookup table.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Apply the callback to each DataFrame in the transformer.

        Args:
            tfm (Transformer): The transformer containing DataFrames to process.
        """
        lut = self._load_lookup_table()
        for grp in tfm.dfs.keys():
            df = tfm.dfs[grp]
            df = self._copy_and_fill_na(df)
            df = self._correct_greater_than(df)  # Ensure to correct 'Value type' if necessary
            df = self._drop_na_rows(df, lut)
            self._perform_lookup(df, lut)
            tfm.dfs[grp] = df  # Update the DataFrame in the transformer

    def _load_lookup_table(self) -> dict:
        """
        Load the lookup table from the Excel file and create a mapping dictionary.

        Returns:
            dict: A dictionary mapping value types to detection limits.
        """
        df = pd.read_excel(self.lut_path)
        df = df.astype({'id': 'int'})
        return dict((v, k) for k, v in df.set_index('id')['name'].to_dict().items())

    def _copy_and_fill_na(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Copy the 'Value type' column and fill NaN values with 'Not Available'.

        Args:
            df (pd.DataFrame): The DataFrame to process.

        Returns:
            pd.DataFrame: The DataFrame with updated 'detection_limit' column.
        """
        df['detection_limit'] = df['Value type']
        df['detection_limit'].fillna('Not Available', inplace=True)
        return df
    
    def _correct_greater_than(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Correct the 'Value type' where it is '>' by changing it to '<'.

        Args:
            df (pd.DataFrame): The DataFrame to process.

        Returns:
            pd.DataFrame: The DataFrame with corrected 'Value type'.
        """
        df.loc[df['detection_limit'] == '>', 'detection_limit'] = '<'
        return df


    def _drop_na_rows(self, df: pd.DataFrame, lut: dict) -> pd.DataFrame:
        """
        Drop rows where the 'detection_limit' column has values not in the lookup table.

        Args:
            df (pd.DataFrame): The DataFrame to process.

        Returns:
            pd.DataFrame: The DataFrame with rows dropped where 'detection_limit' is not in the lookup table.
        """
        return df[df['detection_limit'].isin(lut.keys())]

    def _perform_lookup(self, df: pd.DataFrame, lut: dict):
        """
        Perform lookup to update the 'detection_limit' column based on the lookup table.

        Args:
            df (pd.DataFrame): The DataFrame to process.
            lut (dict): The lookup table dictionary.
        """
        df['detection_limit'] = df['detection_limit'].apply(lambda x: lut.get(x, 0))


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            LookupUnitCB(renaming_unit_rules),
                            LookupDetectionLimitCB(detection_limit_lut_path()),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
tfm.dfs['seawater'][['detection_limit','Value type']]

                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15314
Number of dropped rows                                     0      0
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 



,detection_limit,Value type
0,2,<
1,2,<
2,2,<
3,2,<
4,2,<
...,...,...
18851,2,<
18852,2,<
18853,1,=
18854,1,=


***

### Include Sample Laboratory code. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variable: Sample Laboratory code is not included.*`*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``samplabcode``*

>  MARIS NetCDF format does not include Sample Laboratory code.

In [ ]:
# | export
class RemapDataProviderSampleIdCB(Callback):
    """Remap 'KEY' column to 'samplabcode' in each DataFrame."""

    def __init__(self):
        """
        Initialize the RemapDataProviderSampleIdCB.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Remap 'KEY' column to 'samplabcode' in the DataFrames.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp in tfm.dfs:
            self._remap_sample_id(tfm.dfs[grp])
    
    def _remap_sample_id(self, df: pd.DataFrame):
        """
        Remap the 'KEY' column to 'samplabcode' in the DataFrame.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
        """
        df['samplabcode'] = df['Sample ID']


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            RemapDataProviderSampleIdCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

print(tfm()['seawater']['samplabcode'].unique())
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

['WNZ 01' 'WNZ 02' 'WNZ 03' ... '21-656' '21-657' '21-654']
                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15314
Number of dropped rows                                     0      0
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 



***

### Include Station

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variable: Station ID is not included.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Station``*

>  MARIS NetCDF format does not include Station ID.

The OSPAR dataset includes look-up in the `Seawater_Station_Dictionary.xlsx` file for Station information. 

In [ ]:
Path(fname_in)

Path('../../_data/accdb/ospar/csv')

In [ ]:
dfs['seawater'].columns

Index(['ID', 'Contracting Party', 'RSC Sub-division', 'Station ID',
       'Sample ID', 'LatD', 'LatM', 'LatS', 'LatDir', 'LongD', 'LongM',
       'LongS', 'LongDir', 'Sample type', 'Sampling depth', 'Sampling date',
       'Nuclide', 'Value type', 'Activity or MDA', 'Uncertainty', 'Unit',
       'Data provider', 'Measurement Comment', 'Sample Comment',
       'Reference Comment'],
      dtype='object')

In [ ]:
# | export
class RemapStationIdCB(Callback):
    """Remap Station ID to MARIS format."""

    def __init__(self):
        """
        Initialize the RemapStationIdCB with no specific parameters.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Iterate through all DataFrames in the transformer object and remap 'STATION' to 'station_id'.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp in tfm.dfs.keys():
            self._remap_station_id(tfm.dfs[grp])

    def _remap_station_id(self, df: pd.DataFrame):
        """
        Remap 'STATION' column to 'station_id' in the given DataFrame.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
        """
        df['station'] = df['Station ID'] + ', ' + df['Contracting Party']

In [ ]:

#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            RemapStationIdCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
#print(tfm.dfs['seawater']['station'].unique())
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15314
Number of dropped rows                                     0      0
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 



***

### Measurement note

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variables: Not included in NetCDF*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variable: ``measurenote``*

In [ ]:
# | export
class RecordMeasurementNoteCB(Callback):
    """Record measurement notes by adding a 'measurenote' column to DataFrames."""
    
    def __init__(self):
        """
        Initialize the RecordMeasurementNoteCB.

        This class does not require additional arguments or setup for initialization.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Apply the callback to each DataFrame in the transformer to add the 'measurenote' column.

        Args:
            tfm (Transformer): The transformer object containing DataFrames to process.
        
        This method iterates over all DataFrames in the transformer and checks for the
        presence of the 'Measurement Comment' column. If found, it copies the values
        to a new 'measurenote' column. If not found, it prints a warning message.
        """
        for grp, df in tfm.dfs.items():
            if 'Measurement Comment' in df.columns:
                self._add_measurementnote(df)
            else:
                print(f"Warning: 'Measurement Comment' column not found in DataFrame for group '{grp}'")

    def _add_measurementnote(self, df: pd.DataFrame):
        """
        Add the 'measurenote' column to the DataFrame by mapping values from 'Measurement Comment'.

        Args:
            df (pd.DataFrame): DataFrame containing the 'Measurement Comment' column.
        
        The 'Measurement Comment' column values are copied to the new 'measurenote' column.
        """
        df['measurenote'] = df['Measurement Comment']


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            RecordMeasurementNoteCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15314
Number of dropped rows                                     0      0
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 



***

### Reference note

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variables: Not included in NetCDF*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variable: ``refnote``* 

In [ ]:
# | export
class RecordRefNoteCB(Callback):
    """Record reference notes by adding a 'refnote' column to DataFrames."""
    
    def __init__(self):
        """
        Initialize the RecordRefNoteCB.

        This class does not require additional arguments or setup for initialization.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Apply the callback to each DataFrame in the transformer to add the 'refnote' column.

        Args:
            tfm (Transformer): The transformer object containing DataFrames to process.
        
        This method iterates over all DataFrames in the transformer and checks for the
        presence of the 'Reference Comment' column. If found, it copies the values
        to a new 'refnote' column. If not found, it prints a warning message.
        """
        for grp, df in tfm.dfs.items():
            if 'Reference Comment' in df.columns:
                self._add_refnote(df)
            else:
                print(f"Warning: 'Reference Comment' column not found in DataFrame for group '{grp}'")

    def _add_refnote(self, df: pd.DataFrame):
        """
        Add the 'refnote' column to the DataFrame by mapping values from 'Reference Comment'.

        Args:
            df (pd.DataFrame): DataFrame containing the 'Reference Comment' column.
        
        The 'Reference Comment' column values are copied to the new 'refnote' column.
        """
        df['refnote'] = df['Reference Comment']


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            RecordRefNoteCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15314
Number of dropped rows                                     0      0
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 



***

### Sample note

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variables: Not included in NetCDF*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variable: ``sampnote``*

In [ ]:
dfs['biota'].columns

Index(['ID', 'Contracting Party', 'RSC Sub-division', 'Station ID',
       'Sample ID', 'LatD', 'LatM', 'LatS', 'LatDir', 'LongD', 'LongM',
       'LongS', 'LongDir', 'Sample type', 'Biological group', 'Species',
       'Body Part', 'Sampling date', 'Nuclide', 'Value type',
       'Activity or MDA', 'Uncertainty', 'Unit', 'Data provider',
       'Measurement Comment', 'Sample Comment', 'Reference Comment'],
      dtype='object')

In [ ]:
# | export
class RecordSampleNoteCB(Callback):
    """Record sample notes by adding a 'sampnote' column to DataFrames."""
    
    def __init__(self):
        """
        Initialize the RecordSampleNoteCB.

        This class does not require additional arguments or setup for initialization.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Apply the callback to each DataFrame in the transformer to add the 'sampnote' column.

        Args:
            tfm (Transformer): The transformer object containing DataFrames to process.
        
        This method iterates over all DataFrames in the transformer and checks for the
        presence of the 'Sample Comment' column. If found, it copies the values
        to a new 'sampnote' column. If not found, it prints a warning message.
        """
        for grp, df in tfm.dfs.items():
            if 'Sample Comment' in df.columns:
                self._add_samplenote(df)
            else:
                print(f"Warning: 'Sample Comment' column not found in DataFrame for group '{grp}'")

    def _add_samplenote(self, df: pd.DataFrame):
        """
        Add the 'sampnote' column to the DataFrame by mapping values from 'Sample Comment'.

        Args:
            df (pd.DataFrame): DataFrame containing the 'Measurement Comment' column.
        
        The 'Sample Comment' column values are copied to the new 'sampnote' column.
        """
        df['sampnote'] = df['Sample Comment']


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            RecordSampleNoteCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15314
Number of dropped rows                                     0      0
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 



***

### Standardize Coordinates

#### Capture Coordinates

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variables: ``lon``  and ``lat``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Longitude`` and ``Latitude``.*

Use decimal degree coordinates if available; otherwise, convert from ``LatD``, ``LatM``, ``LatS``, ``LongD``, ``LongM`` and ``LongS``.

In [ ]:
# | export
class ConvertLonLatCB(Callback):
    """Convert Longitude and Latitude values to decimal degrees (DDD.DDDDD°). This class processes DataFrames to convert latitude and longitude from degrees, minutes, and seconds 
    (DMS) format with direction indicators to decimal degrees format."""
    def __init__(self):
        """
        Initialize the ConvertLonLatCB class.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Apply the conversion to latitude and longitude in each DataFrame within the transformer.

        Args:
            tfm (Transformer): The transformer object containing DataFrames to process.
        
        This method processes each DataFrame to convert latitude and longitude values into decimal degrees.
        """
        for grp, df in tfm.dfs.items():
            df['lat'] = self._convert_latitude(df)
            df['lon'] = self._convert_longitude(df)

    def _convert_latitude(self, df: pd.DataFrame) -> pd.Series:
        """
        Convert latitude values from DMS format to decimal degrees.

        Args:
            df (pd.DataFrame): DataFrame containing latitude columns.

        Returns:
            pd.Series: Series with latitude values converted to decimal degrees.
        """
        return np.where(
            df['LatDir'].isin(['S']),
            self._dms_to_decimal(df['LatD'], df['LatM'], df['LatS']) * -1,
            self._dms_to_decimal(df['LatD'], df['LatM'], df['LatS'])
        )

    def _convert_longitude(self, df: pd.DataFrame) -> pd.Series:
        """
        Convert longitude values from DMS format to decimal degrees.

        Args:
            df (pd.DataFrame): DataFrame containing longitude columns.

        Returns:
            pd.Series: Series with longitude values converted to decimal degrees.
        """
        return np.where(
            df['LongDir'].isin(['W']),
            self._dms_to_decimal(df['LongD'], df['LongM'], df['LongS']) * -1,
            self._dms_to_decimal(df['LongD'], df['LongM'], df['LongS'])
        )

    def _dms_to_decimal(self, degrees: pd.Series, minutes: pd.Series, seconds: pd.Series) -> pd.Series:
        """
        Convert DMS (degrees, minutes, seconds) format to decimal degrees.

        Args:
            degrees (pd.Series): Series containing degree values.
            minutes (pd.Series): Series containing minute values.
            seconds (pd.Series): Series containing second values.

        Returns:
            pd.Series: Series with values converted to decimal degrees.
        """
        return degrees + minutes / 60 + seconds / 3600


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            ConvertLonLatCB()
                            ])
tfm()
tfm.dfs['seawater'][['lat','LatD', 'LatM', 'LatS', 'lon', 'LatDir', 'LongD', 'LongM','LongS', 'LongDir']]

,lat,LatD,LatM,LatS,lon,LatDir,LongD,LongM,LongS,LongDir
0,51.375278,51.0,22.0,31.0,3.188056,N,3.0,11.0,17.0,E
1,51.223611,51.0,13.0,25.0,2.859444,N,2.0,51.0,34.0,E
2,51.184444,51.0,11.0,4.0,2.713611,N,2.0,42.0,49.0,E
3,51.420278,51.0,25.0,13.0,3.262222,N,3.0,15.0,44.0,E
4,51.416111,51.0,24.0,58.0,2.809722,N,2.0,48.0,35.0,E
...,...,...,...,...,...,...,...,...,...,...
18851,56.011111,56.0,0.0,40.0,-3.406667,N,3.0,24.0,24.0,W
18852,56.011111,56.0,0.0,40.0,-3.406667,N,3.0,24.0,24.0,W
18853,53.413333,53.0,24.0,48.0,-3.870278,N,3.0,52.0,13.0,W
18854,53.569722,53.0,34.0,11.0,-3.769722,N,3.0,46.0,11.0,W


***

#### Sanitize coordinates

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF format variables: ``lon``  and ``lat``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Longitude decimal`` and ``Latitude decimal``.*

Sanitize coordinates drops a row when both longitude & latitude equal 0 or data contains unrealistic longitude & latitude values. Converts longitude & latitude `,` separator to `.` separator."

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            ConvertLonLatCB(),
                            SanitizeLonLatCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['biota'][['lat','lon']])


                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18856  15314
Number of dropped rows                                     0      0
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 

             lat       lon
0      55.725278 -4.901944
1      54.968889 -3.240556
2      58.565833 -3.791389
3      58.618611 -3.647778
4      55.964722 -2.398056
...          ...       ...
15309  54.455000 -3.566111
15310  48.832778 -1.591389
15311  48.832778 -1.591389
15312  49.551667 -1.860000
15313  49.714444 -1.946111

[15314 rows x 2 columns]


***

### Combine Callbacks and review DFS and TFM data

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            GetSampleTypeCB(type_lut),
                            LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),        
                            SanitizeValueCB(),                       
                            NormalizeUncCB(unc_exp2stan),
                            LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                            CorrectWholeBodyPartCB(),
                            LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupTaxonInformationCB(partial(get_taxon_info_lut, species_lut_path())),
                            LookupUnitCB(renaming_unit_rules),
                            LookupDetectionLimitCB(detection_limit_lut_path()),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RecordMeasurementNoteCB(),
                            RecordRefNoteCB(),
                            RecordSampleNoteCB(),   
                            ConvertLonLatCB(),                    
                            SanitizeLonLatCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18308  15308
Number of dropped rows                                   548      6
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 



In [ ]:
seawater_dfs_dropped_review=tfm.dfs_dropped['seawater']
biota_dfs_dropped_review=tfm.dfs_dropped['biota']

***

### Rename columns of interest for NetCDF or Open Refine/

In [ ]:
tfm.dfs['seawater'].columns

Index(['ID', 'Contracting Party', 'RSC Sub-division', 'Station ID',
       'Sample ID', 'LatD', 'LatM', 'LatS', 'LatDir', 'LongD', 'LongM',
       'LongS', 'LongDir', 'Sample type', 'Sampling depth', 'Sampling date',
       'Nuclide', 'Value type', 'Activity or MDA', 'Uncertainty', 'Unit',
       'Data provider', 'Measurement Comment', 'Sample Comment',
       'Reference Comment', 'samptype_id', 'NUCLIDE', 'nuclide_id', 'time',
       'begperiod', 'value', 'uncertainty', 'unit', 'detection_limit',
       'samplabcode', 'station', 'measurenote', 'refnote', 'sampnote', 'lat',
       'lon'],
      dtype='object')

In [ ]:
tfm.dfs['biota'].columns

Index(['ID', 'Contracting Party', 'RSC Sub-division', 'Station ID',
       'Sample ID', 'LatD', 'LatM', 'LatS', 'LatDir', 'LongD', 'LongM',
       'LongS', 'LongDir', 'Sample type', 'Biological group', 'Species',
       'Body Part', 'Sampling date', 'Nuclide', 'Value type',
       'Activity or MDA', 'Uncertainty', 'Unit', 'Data provider',
       'Measurement Comment', 'Sample Comment', 'Reference Comment',
       'samptype_id', 'NUCLIDE', 'nuclide_id', 'time', 'begperiod', 'value',
       'uncertainty', 'species', 'body_part', 'bio_group', 'TaxonRepName',
       'Taxonname', 'Taxonrank', 'TaxonDB', 'TaxonDBID', 'TaxonDBURL', 'unit',
       'detection_limit', 'samplabcode', 'station', 'measurenote', 'refnote',
       'sampnote', 'lat', 'lon'],
      dtype='object')

In [ ]:
#| export
# Define columns of interest (keys) and renaming rules (values).
def get_renaming_rules(encoding_type='netcdf'):
    vars = cdl_cfg()['vars']
    if encoding_type == 'netcdf':
        return OrderedDict({
            ('seawater', 'biota', 'sediment'): {
                # DEFAULT
                'lat': vars['defaults']['lat']['name'],
                'lon': vars['defaults']['lon']['name'],
                'time': vars['defaults']['time']['name'],
                'NUCLIDE': 'nuclide',
                'detection_limit': vars['suffixes']['detection_limit']['name'],
                'unit': vars['suffixes']['unit']['name'],
                'value': 'value',
                'uncertainty': vars['suffixes']['uncertainty']['name'],
                #'counting_method': vars['suffixes']['counting_method']['name'],
                #'sampling_method': vars['suffixes']['sampling_method']['name'],
                #'preparation_method': vars['suffixes']['preparation_method']['name']
            },
            ('seawater',): {
                # SEAWATER
            },
            ('biota',): {
                # BIOTA
                'species': vars['bio']['species']['name'],
                'body_part': vars['bio']['body_part']['name'],
                'bio_group': vars['bio']['bio_group']['name']
            }
        })
    
    elif encoding_type == 'openrefine':
        return OrderedDict({
            ('seawater', 'biota', 'sediment'): {
                # DEFAULT
                'samptype_id': 'samptype_id',
                'lat': 'latitude',
                'lon': 'longitude',
                'station': 'station',
                'begperiod': 'begperiod',
                'samplabcode': 'samplabcode',
                #'endperiod': 'endperiod',
                'nuclide_id': 'nuclide_id',
                'detection_limit': 'detection',
                'unit': 'unit_id',
                'value': 'activity',
                'uncertainty': 'uncertaint',
                'sampnote': 'sampnote',
                'measurenote': 'measurenote',
                'refnote' : 'refnote'
            },
            ('seawater',) : {
                # SEAWATER
                #'volume': 'volume',
                #'filtpore': 'filtpore',
                #'acid': 'acid'
            },
            ('biota',) : {
                # BIOTA
                'species': 'species_id',
                'Taxonname': 'Taxonname',
                'TaxonRepName': 'TaxonRepName',
                #'Commonname': 'Commonname',
                'Taxonrank': 'Taxonrank',
                'TaxonDB': 'TaxonDB',
                'TaxonDBID': 'TaxonDBID',
                'TaxonDBURL': 'TaxonDBURL',
                'body_part': 'bodypar_id',
            }
        })
    else:
        print("Invalid encoding_type provided. Please use 'netcdf' or 'openrefine'.")
        return None

In [ ]:
#| export
class SelectAndRenameColumnCB(Callback):
    """A callback to select and rename columns in a DataFrame based on provided renaming rules
    for a specified encoding type. It also prints renaming rules that were not applied
    because their keys were not found in the DataFrame."""
    
    def __init__(self, fn_renaming_rules, encoding_type='netcdf', verbose=False):
        """
        Initialize the SelectAndRenameColumnCB callback.

        Args:
            fn_renaming_rules (function): A function that returns an OrderedDict of renaming rules.
            encoding_type (str): The encoding type ('netcdf' or 'openrefine') to determine which renaming rules to use.
            verbose (bool): Whether to print out renaming rules that were not applied.
        """
        fc.store_attr()

    def __call__(self, tfm):
        """
        Apply column selection and renaming to DataFrames in the transformer, and identify unused rules.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        try:
            renaming_rules = self.fn_renaming_rules(self.encoding_type)
        except ValueError as e:
            print(f"Error fetching renaming rules: {e}")
            return

        for group in tfm.dfs.keys():
            # Get relevant renaming rules for the current group
            group_rules = self._get_group_rules(renaming_rules, group)

            if not group_rules:
                continue

            # Apply renaming rules and track keys not found in the DataFrame
            df = tfm.dfs[group]
            df, not_found_keys = self._apply_renaming(df, group_rules)
            tfm.dfs[group] = df
            
            # Print any renaming rules that were not used
            if not_found_keys and self.verbose:
                print(f"\nGroup '{group}' has the following renaming rules not applied:")
                for old_col in not_found_keys:
                    print(f"Key '{old_col}' from renaming rules was not found in the DataFrame.")

    def _get_group_rules(self, renaming_rules, group):
        """
        Retrieve and merge renaming rules for the specified group based on the encoding type.

        Args:
            renaming_rules (OrderedDict): OrderedDict of all renaming rules.
            group (str): Group name to filter rules.

        Returns:
            OrderedDict: An OrderedDict of renaming rules applicable to the specified group.
        """
        relevant_rules = [rules for key, rules in renaming_rules.items() if group in key]
        merged_rules = OrderedDict()
        for rules in relevant_rules:
            merged_rules.update(rules)
        return merged_rules

    def _apply_renaming(self, df, rename_rules):
        """
        Select columns based on renaming rules and apply renaming, only for existing columns,
        while maintaining the order of the dictionary columns.

        Args:
            df (pd.DataFrame): DataFrame to modify.
            rename_rules (OrderedDict): OrderedDict of column renaming rules.

        Returns:
            tuple: A tuple containing:
                - The DataFrame with columns renamed and filtered.
                - A set of column names from renaming rules that were not found in the DataFrame.
        """
        existing_columns = set(df.columns)
        valid_rules = OrderedDict((old_col, new_col) for old_col, new_col in rename_rules.items() if old_col in existing_columns)

        # Create a list to maintain the order of columns
        columns_to_keep = [col for col in rename_rules.keys() if col in existing_columns]
        columns_to_keep += [new_col for old_col, new_col in valid_rules.items() if new_col in df.columns]

        df = df[list(OrderedDict.fromkeys(columns_to_keep))]

        # Apply renaming
        df.rename(columns=valid_rules, inplace=True)

        # Determine which keys were not found
        not_found_keys = set(rename_rules.keys()) - existing_columns
        return df, not_found_keys


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            GetSampleTypeCB(type_lut),
                            LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),        
                            SanitizeValueCB(),                       
                            NormalizeUncCB(unc_exp2stan),
                            LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                            CorrectWholeBodyPartCB(),
                            LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupTaxonInformationCB(partial(get_taxon_info_lut, species_lut_path())),
                            LookupUnitCB(renaming_unit_rules),
                            LookupDetectionLimitCB(detection_limit_lut_path()),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RecordMeasurementNoteCB(),
                            RecordRefNoteCB(),
                            RecordSampleNoteCB(),   
                            ConvertLonLatCB(),                    
                            SanitizeLonLatCB(),
                            SelectAndRenameColumnCB(get_renaming_rules, encoding_type='netcdf'),
                            ])

tfm()
print(tfm.dfs['seawater'].columns)
print(tfm.dfs['biota'].columns)

Index(['lat', 'lon', 'time', 'nuclide', '_dl', '_unit', 'value', '_unc'], dtype='object')
Index(['lat', 'lon', 'time', 'nuclide', '_dl', '_unit', 'value', '_unc',
       'species', 'body_part', 'bio_group'],
      dtype='object')


***

### Reshape: long to wide

Convert data from long to wide and rename columns to comply with NetCDF format.

In [ ]:

#| export
class ReshapeLongToWide(Callback):
    "Convert data from long to wide with renamed columns."
    def __init__(self, columns=['nuclide'], values=['value']):
        fc.store_attr()
        # Retrieve all possible derived vars (e.g 'unc', 'dl', ...) from configs
        self.derived_cols = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
    
    def renamed_cols(self, cols):
        "Flatten columns name"
        return [inner if outer == "value" else f'{inner}{outer}'
                if inner else outer
                for outer, inner in cols]

    def pivot(self, df):
        # Among all possible 'derived cols' select the ones present in df
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        df.index.name = 'org_index'
        df=df.reset_index()
        idx = list(set(df.columns) - set(self.columns + derived_coi + self.values))
        
        # Create a fill_value to replace NaN values in the columns used as the index in the pivot table.
        # Check if num_fill_value is already in the dataframe index values. If num_fill_value already exists
        # then increase num_fill_value by 1 until a value is found for num_fill_value that is not in the dataframe. 
        num_fill_value = -999
        while (df[idx] == num_fill_value).any().any():
            num_fill_value += 1
        # Fill in nan values for each col found in idx. 
        for col in idx:   
            if pd.api.types.is_numeric_dtype(df[col]):
                fill_value = num_fill_value
            if pd.api.types.is_string_dtype(df[col]):
                fill_value = 'NOT AVAILABLE'
                
            df[col]=df[col].fillna(fill_value)

        pivot_df=df.pivot_table(index=idx,
                              columns=self.columns,
                              values=self.values + derived_coi,
                              fill_value=np.nan,
                              aggfunc=lambda x: x
                              ).reset_index()
        

        # Replace fill_value  with  np.nan
        pivot_df[idx]=pivot_df[idx].replace({'NOT AVAILABLE': np.nan,
                                             num_fill_value : np.nan})
        # Set the index to be the org_index
        pivot_df = pivot_df.set_index('org_index')
                
        return (pivot_df)

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            tfm.dfs[grp] = self.pivot(tfm.dfs[grp])
            tfm.dfs[grp].columns = self.renamed_cols(tfm.dfs[grp].columns)

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            GetSampleTypeCB(type_lut),
                            LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),        
                            SanitizeValueCB(),                       
                            NormalizeUncCB(unc_exp2stan),
                            LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                            CorrectWholeBodyPartCB(),
                            LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupTaxonInformationCB(partial(get_taxon_info_lut, species_lut_path())),
                            LookupUnitCB(renaming_unit_rules),
                            LookupDetectionLimitCB(detection_limit_lut_path()),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RecordMeasurementNoteCB(),
                            RecordRefNoteCB(),
                            RecordSampleNoteCB(),   
                            ConvertLonLatCB(),                    
                            SanitizeLonLatCB(),
                            SelectAndRenameColumnCB(get_renaming_rules, encoding_type='netcdf'),
                            ReshapeLongToWide(), 
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(tfm.dfs['biota'].head())
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

                 lon  species        time  bio_group        lat  body_part  \
org_index                                                                    
11474     -39.634444       99  1017014400          4  66.784167         52   
4714      -39.150000      426  1375228800          4  62.116667         52   
6465      -35.920000       99  1287273600          4  64.289722         52   
6466      -35.100000      381  1287100800          4  64.720000         52   
6143      -34.000000       99  1306886400          4  64.000000         52   

           am241_dl  cs134_dl  cs137_dl  h3_dl  ...  cs134   cs137  h3  pb210  \
org_index                                       ...                             
11474           NaN       NaN       1.0    NaN  ...    NaN  0.1800 NaN    NaN   
4714            NaN       NaN       1.0    NaN  ...    NaN  0.2198 NaN    NaN   
6465            NaN       NaN       1.0    NaN  ...    NaN  0.2090 NaN    NaN   
6466            NaN       NaN       1.0    NaN  

In [ ]:
seawater_dfs_review=tfm.dfs['seawater']
biota_dfs_review=tfm.dfs['biota']

***

## NetCDF encoder

### Example change logs

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[                         
                            GetSampleTypeCB(type_lut),
                            LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),        
                            SanitizeValueCB(),                       
                            NormalizeUncCB(unc_exp2stan),
                            LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                            CorrectWholeBodyPartCB(),
                            LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupTaxonInformationCB(partial(get_taxon_info_lut, species_lut_path())),
                            LookupUnitCB(renaming_unit_rules),
                            LookupDetectionLimitCB(detection_limit_lut_path()),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RecordMeasurementNoteCB(),
                            RecordRefNoteCB(),
                            RecordSampleNoteCB(),   
                            ConvertLonLatCB(),                    
                            SanitizeLonLatCB(),
                            SelectAndRenameColumnCB(get_renaming_rules, encoding_type='netcdf'),
                            ReshapeLongToWide(), 
                            CompareDfsAndTfmCB(dfs)
                            ])

# Transform
tfm()
# Check transformation logs
tfm.logs

["Set the 'Sample type' column in the DataFrames based on a lookup table.",
 'Convert nuclide names to lowercase and strip any trailing spaces.',
 'Remap and standardize radionuclide names to MARIS radionuclide names and define nuclide ids.',
 'Encode time as `int` representing seconds since xxx',
 'Sanitize value by removing blank entries.',
 'Callback to normalize uncertainty values in DataFrames. This callback applies a conversion function to standardize the uncertainty values in each DataFrame.',
 "Remap biota species to MARIS database format.This class updates the 'Species' column in the biota DataFrame by:\n    - Replacing 'NaN' or 'Not available' values with corresponding biological groups.\n    - Performing a lookup to remap species to MARIS format.",
 "Update body parts labeled as 'whole' to either 'Whole animal' or 'Whole plant'.",
 'Update body part id based on MARIS dbo_bodypar.xlsx',
 'Update biogroup id based on MARIS dbo_species.xlsx.',
 'Update taxon names based on MARI

***

### Feed global attributes

In [ ]:
#| export
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']


In [ ]:
#| export
def get_attrs(tfm, zotero_key, kw=kw):
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(cfg()),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
#|eval: false
get_attrs(tfm, zotero_key=zotero_key, kw=kw)

{'geospatial_lat_min': '49.43222222222222',
 'geospatial_lat_max': '81.26805555555555',
 'geospatial_lon_min': '-58.23166666666667',
 'geospatial_lon_max': '36.181666666666665',
 'geospatial_bounds': 'POLYGON ((-58.23166666666667 36.181666666666665, 49.43222222222222 36.181666666666665, 49.43222222222222 81.26805555555555, -58.23166666666667 81.26805555555555, -58.23166666666667 36.181666666666665))',
 'time_coverage_start': '1995-01-01T00:00:00',
 'time_coverage_end': '2021-12-31T00:00:00',
 'title': 'OSPAR Environmental Monitoring of Radioactive Substances',
 'summary': '',
 'creator_name': '[{"creatorType": "author", "firstName": "", "lastName": "OSPAR Comission\'s Radioactive Substances Committee (RSC)"}]',
 'keywords': 'oceanography, Earth Science > Oceans > Ocean Chemistry> Radionuclides, Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure, Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments, Earth 

In [ ]:
#| export
def enums_xtra(tfm, vars):
    "Retrieve a subset of the lengthy enum as 'species_t' for instance"
    enums = Enums(lut_src_dir=lut_path(), cdl_enums=cdl_cfg()['enums'])
    xtras = {}
    for var in vars:
        unique_vals = tfm.unique(var)
        if unique_vals.any():
            xtras[f'{var}_t'] = enums.filter(f'{var}_t', unique_vals)
    return xtras

### Encoding NETCDF

In [ ]:
#| export
def encode(fname_in, fname_out_nc, nc_tpl_path, **kwargs):
    dfs = load_data(fname_in)
    tfm = Transformer(dfs, cbs=[
                                GetSampleTypeCB(type_lut),
                                LowerStripRdnNameCB(),
                                RemapRdnNameCB(),
                                ParseTimeCB(),
                                EncodeTimeCB(cfg()),        
                                SanitizeValueCB(),                       
                                NormalizeUncCB(unc_exp2stan),
                                LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                                CorrectWholeBodyPartCB(),
                                LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                                LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                                LookupTaxonInformationCB(partial(get_taxon_info_lut, species_lut_path())),
                                LookupUnitCB(renaming_unit_rules),
                                LookupDetectionLimitCB(detection_limit_lut_path()),
                                RemapDataProviderSampleIdCB(),
                                RemapStationIdCB(),
                                RecordMeasurementNoteCB(),
                                RecordRefNoteCB(),
                                RecordSampleNoteCB(),   
                                ConvertLonLatCB(),                    
                                SanitizeLonLatCB(),
                                SelectAndRenameColumnCB(get_renaming_rules, encoding_type='netcdf'),
                                ReshapeLongToWide(),
                                ])
    tfm()
    encoder = NetCDFEncoder(tfm.dfs, 
                            src_fname=nc_tpl_path,
                            dest_fname=fname_out_nc, 
                            global_attrs=get_attrs(tfm, zotero_key=zotero_key, kw=kw),
                            verbose=kwargs.get('verbose', False),
                            enums_xtra=enums_xtra(tfm, vars=['species', 'body_part'])
                           )
    encoder.encode()

In [ ]:
#|eval: false
encode(fname_in, fname_out_nc, nc_tpl_path(), verbose=True)

--------------------------------------------------------------------------------
Group: seawater, Variable: lon
--------------------------------------------------------------------------------
Group: seawater, Variable: lat
--------------------------------------------------------------------------------
Group: seawater, Variable: time
--------------------------------------------------------------------------------
Group: seawater, Variable: h3
--------------------------------------------------------------------------------
Group: seawater, Variable: h3_unc
--------------------------------------------------------------------------------
Group: seawater, Variable: h3_dl
--------------------------------------------------------------------------------
Group: seawater, Variable: h3_unit
--------------------------------------------------------------------------------
Group: seawater, Variable: tc99
--------------------------------------------------------------------------------
Group: seawat

***

## Open Refine Pipeline

### Rename columns for Open Refine

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            GetSampleTypeCB(type_lut),
                            LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),        
                            SanitizeValueCB(),                       
                            NormalizeUncCB(unc_exp2stan),
                            LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                            CorrectWholeBodyPartCB(),
                            LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupTaxonInformationCB(partial(get_taxon_info_lut, species_lut_path())),
                            LookupUnitCB(renaming_unit_rules),
                            LookupDetectionLimitCB(detection_limit_lut_path()),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RecordMeasurementNoteCB(),
                            RecordRefNoteCB(),
                            RecordSampleNoteCB(),   
                            ConvertLonLatCB(),                    
                            SanitizeLonLatCB(),
                            SelectAndRenameColumnCB(get_renaming_rules, encoding_type='openrefine', verbose=True),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

                                                    seawater  biota
Number of rows in dfs                                  18856  15314
Number of rows in tfm.dfs                              18308  15308
Number of dropped rows                                   548      6
Number of rows in tfm.dfs + Number of dropped rows     18856  15314 



**Example of data included in dfs_dropped.**

Main reasons for data to be dropped from dfs:
- No activity value reported (i.e. ``Activity or MDA``)

Reason 6 biota values are dropped:
- The body part is not known (i.e.'Mix of muscle and whole fish without liver' or 'UNKNOWN') 

In [ ]:
grp='seawater'
#grp='biota'
tfm.dfs_dropped[grp]

,ID,Contracting Party,RSC Sub-division,Station ID,Sample ID,LatD,LatM,LatS,LatDir,LongD,...,Sampling date,Nuclide,Value type,Activity or MDA,Uncertainty,Unit,Data provider,Measurement Comment,Sample Comment,Reference Comment
16799,97147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16800,97148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16801,97149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16802,97150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16803,97151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18474,120366,Ireland,4.0,N8,NaN,53.0,39.0,0.0,N,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021 data,NaN,NaN
18475,120367,Ireland,4.0,N9,NaN,53.0,53.0,0.0,N,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021 data,NaN,NaN
18476,120368,Ireland,4.0,N10,NaN,53.0,52.0,0.0,N,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021 data,NaN,NaN
18477,120369,Ireland,1.0,Salthill,NaN,53.0,15.0,40.0,N,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021 data,Woodstown (County Waterford) and Salthill (Cou...,NaN


## Open Refine encoder

In [ ]:
#| export
def encode_or(fname_in, fname_out_csv, ref_id, **kwargs):
    dfs = load_data(fname_in)
    tfm = Transformer(dfs, cbs=[
                                GetSampleTypeCB(type_lut),
                                LowerStripRdnNameCB(),
                                RemapRdnNameCB(),
                                ParseTimeCB(),
                                EncodeTimeCB(cfg()),        
                                SanitizeValueCB(),                       
                                NormalizeUncCB(unc_exp2stan),
                                LookupBiotaSpeciesCB(get_maris_species, unmatched_fixes_biota_species),
                                CorrectWholeBodyPartCB(),
                                LookupBiotaBodyPartCB(get_maris_bodypart, unmatched_fixes_biota_tissues),
                                LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                                LookupTaxonInformationCB(partial(get_taxon_info_lut, species_lut_path())),
                                LookupUnitCB(renaming_unit_rules),
                                LookupDetectionLimitCB(detection_limit_lut_path()),
                                RemapDataProviderSampleIdCB(),
                                RemapStationIdCB(),
                                RecordMeasurementNoteCB(),
                                RecordRefNoteCB(),
                                RecordSampleNoteCB(),   
                                ConvertLonLatCB(),                    
                                SanitizeLonLatCB(),
                                SelectAndRenameColumnCB(get_renaming_rules, encoding_type='openrefine', verbose=True),
                                CompareDfsAndTfmCB(dfs)
                                ])
    tfm()

    encoder = OpenRefineCsvEncoder(tfm.dfs, 
                                    dest_fname=fname_out_csv, 
                                    ref_id = ref_id,
                                    verbose = True
                                )
    encoder.encode()

In [ ]:
#|eval: false
encode_or(fname_in, fname_out_csv, ref_id, verbose=True)